In [99]:
import numpy as np
import pandas as pd
from pathlib import Path


In [74]:
import sys
sys.path.insert(0, "..")   
from src.utils import (
generate_cyclical_features, 
generate_sales_features, 
add_y_targets_from_shift,
prepare_training_data_from_raw_df,
build_feature_and_label_cols,
)

In [75]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
dtype_dict={"id":np.uint32,
            "store_nbr":np.uint8,
            "item_nbr":np.uint32,
            "unit_sales":np.float32
           }
df = pd.read_csv("../output/data/20250630_train_top_store_500_item.csv", dtype=dtype_dict)
df.rename(columns={"store_nbr": "store", "item_nbr": "item"}, inplace=True)
df["store_item"] = df["store"].astype(str) + "_" + df["item"].astype(str)
cols = ['date', 'store_item', 'store', 'item'] + [c for c in df.columns 
                                  if c not in ('date', 'store_item', 'store', 'item')]
df = df[cols]
df["date"] = pd.to_datetime(df["date"])
df.sort_values("date", inplace=True)
df.reset_index(drop=True, inplace=True)
df.head(5)

/var/folders/w3/d06m5l_97yn4bnxd5t6cgbjr0000gn/T/ipykernel_54173/1720511107.py:6: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../output/data/20250630_train_top_store_500_item.csv", dtype=dtype_dict)


,date,store_item,store,item,id,unit_sales,onpromotion
0,2013-01-01,25_108786,25,108786,5,3.0,NaN
1,2013-01-01,25_769314,25,769314,355,3.0,NaN
2,2013-01-01,25_771156,25,771156,356,1.0,NaN
3,2013-01-01,25_783243,25,783243,362,5.0,NaN
4,2013-01-01,25_801217,25,801217,368,3.0,NaN


In [33]:
df.query("store_item == '54_1254013'").to_csv("../output/data/df_54_1254013.csv", index=False)

In [41]:
cluster_df = pd.read_csv("../output/data/20250629_cluster_df.csv")
cluster_df


,store,item,store_cluster,item_cluster
0,16,215352,0,0
1,16,220435,0,0
2,16,222879,0,0
3,16,265559,0,0
4,16,273528,0,0
...,...,...,...,...
26995,48,1149579,2,7
26996,48,1157561,2,7
26997,48,1165988,2,7
26998,48,1165989,2,7


In [78]:
sales_df = generate_sales_features(df.query("store_item == '54_1254013'"),
                                   window_size=16,
                                   cluster_df=cluster_df,
                                   calendar_aligned=True,
                                   debug=True,
                                   debug_fn="../output/data/20250630_debug_cluster_mappings.csv")   
sales_df.head()


Window 2014-04-02: 100%|██████████| 1/1 [00:00<00:00, 388.54it/s]
Window 2014-04-18: 0it [00:00, ?it/s]
Window 2017-07-31: 100%|██████████| 1/1 [00:00<00:00, 313.48it/s]


,start_date,id,store_item,store,item,storeClusterId,itemClusterId,sales_day_1,sales_day_2,sales_day_3,...,item_med_day_7,item_med_day_8,item_med_day_9,item_med_day_10,item_med_day_11,item_med_day_12,item_med_day_13,item_med_day_14,item_med_day_15,item_med_day_16
0,2014-03-01,19725775,54_1254013,54,1254013,0,6,3.0,27.0,27.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2014-03-01,19790294,54_1254013,54,1254013,0,6,3.0,27.0,27.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2014-03-01,19852836,54_1254013,54,1254013,0,6,3.0,27.0,27.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,2014-03-01,19917016,54_1254013,54,1254013,0,6,3.0,27.0,27.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2014-03-01,19982870,54_1254013,54,1254013,0,6,3.0,27.0,27.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [79]:
sales_df.columns.to_list()

['start_date',
 'id',
 'store_item',
 'store',
 'item',
 'storeClusterId',
 'itemClusterId',
 'sales_day_1',
 'sales_day_2',
 'sales_day_3',
 'sales_day_4',
 'sales_day_5',
 'sales_day_6',
 'sales_day_7',
 'sales_day_8',
 'sales_day_9',
 'sales_day_10',
 'sales_day_11',
 'sales_day_12',
 'sales_day_13',
 'sales_day_14',
 'sales_day_15',
 'sales_day_16',
 'store_med_day_1',
 'store_med_day_2',
 'store_med_day_3',
 'store_med_day_4',
 'store_med_day_5',
 'store_med_day_6',
 'store_med_day_7',
 'store_med_day_8',
 'store_med_day_9',
 'store_med_day_10',
 'store_med_day_11',
 'store_med_day_12',
 'store_med_day_13',
 'store_med_day_14',
 'store_med_day_15',
 'store_med_day_16',
 'item_med_day_1',
 'item_med_day_2',
 'item_med_day_3',
 'item_med_day_4',
 'item_med_day_5',
 'item_med_day_6',
 'item_med_day_7',
 'item_med_day_8',
 'item_med_day_9',
 'item_med_day_10',
 'item_med_day_11',
 'item_med_day_12',
 'item_med_day_13',
 'item_med_day_14',
 'item_med_day_15',
 'item_med_day_16']

In [80]:
sales_df.shape

(93132, 55)

In [81]:
cyc_df = generate_cyclical_features(df.query("store_item == '54_1254013'"), 
                                    window_size=16,
                                    calendar_aligned=True,
                                    cluster_df=cluster_df,
                                    debug=True,
                                    debug_fn="../output/data/20250629_debug_cluster_mappings.csv")
cyc_df.head()

Generating cyclical features: 100%|██████████| 1/1 [00:00<00:00,  3.48it/s]


,start_date,id,store_item,store,item,storeClusterId,itemClusterId,dayofweek_sin_1,dayofweek_cos_1,weekofmonth_sin_1,...,dayofweek_sin_16,dayofweek_cos_16,weekofmonth_sin_16,weekofmonth_cos_16,monthofyear_sin_16,monthofyear_cos_16,paycycle_sin_16,paycycle_cos_16,season_sin_16,season_cos_16
0,2014-03-01,19725775,54_1254013,54,1254013,0,6,-0.974928,-0.222521,0.951057,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2014-03-01,19790294,54_1254013,54,1254013,0,6,-0.974928,-0.222521,0.951057,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2014-03-01,19852836,54_1254013,54,1254013,0,6,-0.974928,-0.222521,0.951057,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2014-03-01,19917016,54_1254013,54,1254013,0,6,-0.974928,-0.222521,0.951057,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2014-03-01,19982870,54_1254013,54,1254013,0,6,-0.974928,-0.222521,0.951057,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [82]:
sales_df.shape

(93132, 55)

In [83]:
cyc_df.shape

(93132, 167)

In [86]:
# Define the keys used for merging
merge_keys = ["start_date", "id", "store_item", "store", "item"]

# Identify missing rows from non_overlap_df2 that are not in non_overlap_df
missing_from_df = sales_df.merge(
    cyc_df[merge_keys],
    on=merge_keys,
    how="left",
    indicator=True
).query('_merge == "left_only"')

# Optional: drop the merge indicator column
missing_from_df = missing_from_df.drop(columns="_merge")

# Show how many are missing
print(f"Missing rows in non_overlap_df: {missing_from_df.shape[0]}")


Missing rows in non_overlap_df: 0


In [95]:
sales_df["start_date"] = pd.to_datetime(sales_df["start_date"])
cyc_df["start_date"] = pd.to_datetime(cyc_df["start_date"])

merged_df = pd.merge(sales_df, cyc_df, on=["store_item", "id","start_date", "store", "item","storeClusterId","itemClusterId" ])

merged_df.head()

,start_date,id,store_item,store,item,storeClusterId,itemClusterId,sales_day_1,sales_day_2,sales_day_3,...,dayofweek_sin_16,dayofweek_cos_16,weekofmonth_sin_16,weekofmonth_cos_16,monthofyear_sin_16,monthofyear_cos_16,paycycle_sin_16,paycycle_cos_16,season_sin_16,season_cos_16
0,2014-03-01,19725775,54_1254013,54,1254013,0,6,3.0,27.0,27.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2014-03-01,19790294,54_1254013,54,1254013,0,6,3.0,27.0,27.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2014-03-01,19852836,54_1254013,54,1254013,0,6,3.0,27.0,27.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2014-03-01,19917016,54_1254013,54,1254013,0,6,3.0,27.0,27.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2014-03-01,19982870,54_1254013,54,1254013,0,6,3.0,27.0,27.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [88]:
merged_df.shape

(93132, 215)

In [96]:
final_df = add_y_targets_from_shift(merged_df, window_size=16)
y_cols = [col for col in final_df.columns if col.startswith("y_")]
final_df = final_df.dropna(subset=y_cols)
final_df.shape


(76, 423)

In [90]:
final_df.head()

,start_date,id,store_item,store,item,storeClusterId,itemClusterId,sales_day_1,sales_day_2,sales_day_3,...,y_dayofweek_sin_16,y_dayofweek_cos_16,y_weekofmonth_sin_16,y_weekofmonth_cos_16,y_monthofyear_sin_16,y_monthofyear_cos_16,y_paycycle_sin_16,y_paycycle_cos_16,y_season_sin_16,y_season_cos_16
1193,2014-03-01,47539078,54_1254013,54,1254013,0,6,3.0,27.0,27.0,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
2387,2014-03-17,47539078,54_1254013,54,1254013,0,6,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
4775,2014-05-04,47472295,54_1254013,54,1254013,0,6,0.0,0.0,0.0,...,0.974928,-0.222521,0.951057,0.309017,1.224647e-16,-1.000000,0.994522,-0.104528,0.965740,0.259512
5969,2014-05-20,47539078,54_1254013,54,1254013,0,6,19.0,13.0,1.0,...,-0.433884,-0.900969,-0.587785,-0.809017,1.224647e-16,-1.000000,0.866025,-0.500000,0.999917,-0.012910
7163,2014-06-05,47606008,54_1254013,54,1254013,0,6,7.0,12.0,19.0,...,-0.781831,0.623490,0.951057,0.309017,-5.000000e-01,-0.866025,0.587785,-0.809017,0.958718,-0.284359


In [26]:
store_item_ids = [3_114790]
df.query("store_item == '3_114790'")

,date,store_item,store,item,unit_sales,onpromotion
11544,2013-01-02,3_114790,3,114790,27.0,NaN
28350,2013-01-03,3_114790,3,114790,19.0,NaN
45392,2013-01-04,3_114790,3,114790,25.0,NaN
62820,2013-01-05,3_114790,3,114790,34.0,NaN
80657,2013-01-06,3_114790,3,114790,38.0,NaN
...,...,...,...,...,...,...
34730878,2017-08-11,3_114790,3,114790,17.0,True
34752592,2017-08-12,3_114790,3,114790,25.0,True
34775129,2017-08-13,3_114790,3,114790,23.0,True
34795781,2017-08-14,3_114790,3,114790,24.0,True


In [25]:
df.head()

,date,store_item,store,item,unit_sales,onpromotion
0,2013-01-01,25_108786,25,108786,3.0,NaN
1,2013-01-01,25_769314,25,769314,3.0,NaN
2,2013-01-01,25_771156,25,771156,1.0,NaN
3,2013-01-01,25_783243,25,783243,5.0,NaN
4,2013-01-01,25_801217,25,801217,3.0,NaN


In [100]:
final_df = prepare_training_data_from_raw_df(
    df.query("store_item == '3_114790'"),
    window_size=16,
    cluster_df=cluster_df,
    calendar_aligned=True,
    debug=True,
    debug_cyc_fn=Path("../output/data/20250630_cyc_debug.csv"),
    sales_fn=Path("../output/data/20250630_sales.csv"),
    cyc_fn=Path("../output/data/20250630_cyc.csv"),
    debug_sales_fn=Path("../output/data/20250630_sales_debug.csv"),
)

2025-06-30 19:00:01,575 - INFO - Loading sales features from ../output/data/20250630_sales.csv
2025-06-30 19:00:02,066 - INFO - sales_df.shape: (93132, 55)
2025-06-30 19:00:02,067 - INFO - Loading cyclical features from ../output/data/20250630_cyc.csv
2025-06-30 19:00:04,684 - INFO - cyc_df.shape: (93132, 167)
2025-06-30 19:00:04,933 - INFO - merged_df.shape: (93132, 215)
2025-06-30 19:00:45,175 - INFO - merged_df.shape: (76, 423)


,start_date,id,store_item,store,item,storeClusterId,itemClusterId,sales_day_1,sales_day_2,sales_day_3,...,y_dayofweek_sin_16,y_dayofweek_cos_16,y_weekofmonth_sin_16,y_weekofmonth_cos_16,y_monthofyear_sin_16,y_monthofyear_cos_16,y_paycycle_sin_16,y_paycycle_cos_16,y_season_sin_16,y_season_cos_16
1193,2014-03-01,47539078,54_1254013,54,1254013,0,6,3.0,27.0,27.0,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
2387,2014-03-17,47539078,54_1254013,54,1254013,0,6,0.0,0.0,0.0,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
4775,2014-05-04,47472295,54_1254013,54,1254013,0,6,0.0,0.0,0.0,...,0.974928,-0.222521,9.510565e-01,0.309017,1.224647e-16,-1.000000,0.994522,-0.104528,0.965740,0.259512
5969,2014-05-20,47539078,54_1254013,54,1254013,0,6,19.0,13.0,1.0,...,-0.433884,-0.900969,-5.877853e-01,-0.809017,1.224647e-16,-1.000000,0.866025,-0.500000,0.999917,-0.012910
7163,2014-06-05,47606008,54_1254013,54,1254013,0,6,7.0,12.0,19.0,...,-0.781831,0.623490,9.510565e-01,0.309017,-5.000000e-01,-0.866025,0.587785,-0.809017,0.958718,-0.284359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87161,2017-05-12,47606008,54_1254013,54,1254013,0,6,2.0,8.0,10.0,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
88355,2017-05-28,47539078,54_1254013,54,1254013,0,6,15.0,7.0,9.0,...,0.974928,-0.222521,-9.510565e-01,0.309017,1.224647e-16,-1.000000,-0.743145,0.669131,0.988678,-0.150055
89549,2017-06-13,47606008,54_1254013,54,1254013,0,6,2.0,1.0,5.0,...,-0.433884,-0.900969,5.877853e-01,-0.809017,-5.000000e-01,-0.866025,-0.406737,0.913545,0.910605,-0.413279
90743,2017-06-29,47606008,54_1254013,54,1254013,0,6,12.0,7.0,7.0,...,-0.781831,0.623490,-2.449294e-16,1.000000,-5.000000e-01,-0.866025,-0.382683,0.923880,0.763889,-0.645348


In [101]:
(
            meta_cols,
            _,
            _,
            x_feature_cols,
            label_cols,
            _,
            _,
) = build_feature_and_label_cols(window_size=16)
final_df[meta_cols + x_feature_cols + label_cols]


,start_date,id,store_item,storeClusterId,itemClusterId,sales_day_1,sales_day_2,sales_day_3,sales_day_4,sales_day_5,...,y_season_cos_7,y_season_cos_8,y_season_cos_9,y_season_cos_10,y_season_cos_11,y_season_cos_12,y_season_cos_13,y_season_cos_14,y_season_cos_15,y_season_cos_16
1193,2014-03-01,47539078,54_1254013,0,6,3.0,27.0,27.0,4.0,8.0,...,0.994671,0.992749,0.990532,0.988023,0.985220,0.982126,0.978740,0.000000,0.000000,0.000000
2387,2014-03-17,47539078,54_1254013,0,6,0.0,0.0,0.0,11.0,15.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4775,2014-05-04,47472295,54_1254013,0,6,0.0,0.0,0.0,0.0,0.0,...,0.405426,0.389630,0.373720,0.357698,0.341571,0.325342,0.309017,0.292600,0.276097,0.259512
5969,2014-05-20,47539078,54_1254013,0,6,19.0,13.0,1.0,7.0,20.0,...,0.141540,0.124479,0.107381,0.090252,0.073095,0.055917,0.038722,0.021516,0.004304,-0.012910
7163,2014-06-05,47606008,54_1254013,0,6,7.0,12.0,19.0,22.0,19.0,...,-0.133015,-0.150055,-0.167052,-0.183998,-0.200891,-0.217723,-0.234491,-0.251190,-0.267814,-0.284359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87161,2017-05-12,47606008,54_1254013,0,6,2.0,8.0,10.0,6.0,6.0,...,0.276097,0.259512,0.242850,0.226116,0.209315,0.192452,0.158559,0.141540,0.124479,0.000000
88355,2017-05-28,47539078,54_1254013,0,6,15.0,7.0,9.0,9.0,12.0,...,0.004304,-0.012910,-0.030120,-0.047321,-0.064508,-0.081676,-0.098820,-0.115935,-0.133015,-0.150055
89549,2017-06-13,47606008,54_1254013,0,6,2.0,1.0,5.0,1.0,4.0,...,-0.267814,-0.284359,-0.300820,-0.317191,-0.333469,-0.349647,-0.365723,-0.381689,-0.397543,-0.413279
90743,2017-06-29,47606008,54_1254013,0,6,12.0,7.0,7.0,12.0,4.0,...,-0.519744,-0.534373,-0.548843,-0.563151,-0.577292,-0.591261,-0.605056,-0.618671,-0.632103,-0.645348


In [17]:
final_df = prepare_training_data_from_raw_df(
    df,
    window_size=16,
    cluster_df=cluster_df,
    calendar_aligned=True,
    debug=True,
    debug_cyc_fn="../output/data/20250629_cyc_debug.csv",
    debug_sales_fn="../output/data/20250629_sales_debug.csv",
)

IndexError: single positional indexer is out-of-bounds

In [ ]:
final_df.head(5)

In [ ]:
meta_cols,\
x_sales_features,\
x_cyclical_features,\
x_feature_cols,\
label_cols,\
y_sales_features,\
y_cyclical_features = build_feature_and_label_cols(window_size=16)
print(meta_cols)
print(x_feature_cols)
print(label_cols)

In [ ]:
final_df[meta_cols + x_feature_cols + label_cols].to_excel("../output/data/20250629_train_store_500_item_sales_cyclical_features_16_days_X_y.xlsx",
            index=False)